In [2]:
from models import *
from config import *

langs = LanguageModel(config)
metar = MetaReasoner(config)

output_seqs = langs([
    "what is that ?",
    "what is Banach-Steinhaus theorem ?",
    "what is that concept ?"])

print(metar.executor.parse("filter(scene(),banach_space)"))
print(metar.executor.get_concept_embedding("banach_space").shape)

for s in output_seqs["seq_features"]:
    print(s.shape)




Filter(Scene(), banach_space)
torch.Size([1, 200])
torch.Size([6, 232])
torch.Size([7, 232])
torch.Size([7, 232])
